# Parsing arguments and building values

These functions are useful when creating your own extensions functions
and methods. Additional information and examples are available in
`extending-index`.

The first three of these functions described, :c`PyArg_ParseTuple`,
:c`PyArg_ParseTupleAndKeywords`, and :c`PyArg_Parse`, all use *format
strings* which are used to tell the function about the expected
arguments. The format strings use the same syntax for each of these
functions.

## Parsing arguments

A format string consists of zero or more "format units." A format unit
describes one Python object; it is usually a single character or a
parenthesized sequence of format units. With a few exceptions, a format
unit that is not a parenthesized sequence normally corresponds to a
single address argument to these functions. In the following
description, the quoted form is the format unit; the entry in (round)
parentheses is the Python object type that matches the format unit; and
the entry in \[square\] brackets is the type of the C variable(s) whose
address should be passed.

### Strings and buffers

These formats allow accessing an object as a contiguous chunk of memory.
You don't have to provide raw storage for the returned unicode or bytes
area.

In general, when a format sets a pointer to a buffer, the buffer is
managed by the corresponding Python object, and the buffer shares the
lifetime of this object. You won't have to release any memory yourself.
The only exceptions are `es`, `es#`, `et` and `et#`.

However, when a :c`Py_buffer` structure gets filled, the underlying
buffer is locked so that the caller can subsequently use the buffer even
inside a :c`Py_BEGIN_ALLOW_THREADS` block without the risk of mutable
data being resized or destroyed. As a result, **you have to call**
:c`PyBuffer_Release` after you have finished processing the data (or in
any early abort case).

Unless otherwise stated, buffers are not NUL-terminated.

Some formats require a read-only `bytes-like object`, and set a pointer
instead of a buffer structure. They work by checking that the object's
:c`PyBufferProcs.bf_releasebuffer` field is `NULL`, which disallows
mutable objects such as `bytearray`.

Note

For all `#` variants of formats (`s#`, `y#`, etc.), the macro
:c`PY_SSIZE_T_CLEAN` must be defined before including `Python.h`. On
Python 3.9 and older, the type of the length argument is :c`Py_ssize_t`
if the :c`PY_SSIZE_T_CLEAN` macro is defined, or int otherwise.

`s` (`str`) \[const char \*\]  
Convert a Unicode object to a C pointer to a character string. A pointer
to an existing string is stored in the character pointer variable whose
address you pass. The C string is NUL-terminated. The Python string must
not contain embedded null code points; if it does, a `ValueError`
exception is raised. Unicode objects are converted to C strings using
`'utf-8'` encoding. If this conversion fails, a `UnicodeError` is
raised.

Note

This format does not accept `bytes-like objects
<bytes-like object>`. If you want to accept filesystem paths and convert
them to C character strings, it is preferable to use the `O&` format
with :c`PyUnicode_FSConverter` as *converter*.

3.5 Previously, `TypeError` was raised when embedded null code points
were encountered in the Python string.

`s*` (`str` or `bytes-like object`) \[Py_buffer\]  
This format accepts Unicode objects as well as bytes-like objects. It
fills a :c`Py_buffer` structure provided by the caller. In this case the
resulting C string may contain embedded NUL bytes. Unicode objects are
converted to C strings using `'utf-8'` encoding.

`s#` (`str`, read-only `bytes-like object`) \[const char \*, :c`Py_ssize_t`\]  
Like `s*`, except that it doesn't accept mutable objects. The result is
stored into two C variables, the first one a pointer to a C string, the
second one its length. The string may contain embedded null bytes.
Unicode objects are converted to C strings using `'utf-8'` encoding.

`z` (`str` or `None`) \[const char \*\]  
Like `s`, but the Python object may also be `None`, in which case the C
pointer is set to `NULL`.

`z*` (`str`, `bytes-like object` or `None`) \[Py_buffer\]  
Like `s*`, but the Python object may also be `None`, in which case the
`buf` member of the :c`Py_buffer` structure is set to `NULL`.

`z#` (`str`, read-only `bytes-like object` or `None`) \[const char \*, :c`Py_ssize_t`\]  
Like `s#`, but the Python object may also be `None`, in which case the C
pointer is set to `NULL`.

`y` (read-only `bytes-like object`) \[const char \*\]  
This format converts a bytes-like object to a C pointer to a character
string; it does not accept Unicode objects. The bytes buffer must not
contain embedded null bytes; if it does, a `ValueError` exception is
raised.

3.5 Previously, `TypeError` was raised when embedded null bytes were
encountered in the bytes buffer.

`y*` (`bytes-like object`) \[Py_buffer\]  
This variant on `s*` doesn't accept Unicode objects, only bytes-like
objects. **This is the recommended way to accept binary data.**

`y#` (read-only `bytes-like object`) \[const char \*, :c`Py_ssize_t`\]  
This variant on `s#` doesn't accept Unicode objects, only bytes-like
objects.

`S` (`bytes`) \[PyBytesObject \*\]  
Requires that the Python object is a `bytes` object, without attempting
any conversion. Raises `TypeError` if the object is not a bytes object.
The C variable may also be declared as :c`PyObject*`.

`Y` (`bytearray`) \[PyByteArrayObject \*\]  
Requires that the Python object is a `bytearray` object, without
attempting any conversion. Raises `TypeError` if the object is not a
`bytearray` object. The C variable may also be declared as
:c`PyObject*`.

`U` (`str`) \[PyObject \*\]  
Requires that the Python object is a Unicode object, without attempting
any conversion. Raises `TypeError` if the object is not a Unicode
object. The C variable may also be declared as :c`PyObject*`.

`w*` (read-write `bytes-like object`) \[Py_buffer\]  
This format accepts any object which implements the read-write buffer
interface. It fills a :c`Py_buffer` structure provided by the caller.
The buffer may contain embedded null bytes. The caller have to call
:c`PyBuffer_Release` when it is done with the buffer.

`es` (`str`) \[const char \*encoding, char \*\*buffer\]  
This variant on `s` is used for encoding Unicode into a character
buffer. It only works for encoded data without embedded NUL bytes.

This format requires two arguments. The first is only used as input, and
must be a :c`const char*` which points to the name of an encoding as a
NUL-terminated string, or `NULL`, in which case `'utf-8'` encoding is
used. An exception is raised if the named encoding is not known to
Python. The second argument must be a :c`char**`; the value of the
pointer it references will be set to a buffer with the contents of the
argument text. The text will be encoded in the encoding specified by the
first argument.

:c`PyArg_ParseTuple` will allocate a buffer of the needed size, copy the
encoded data into this buffer and adjust *\*buffer* to reference the
newly allocated storage. The caller is responsible for calling
:c`PyMem_Free` to free the allocated buffer after use.

`et` (`str`, `bytes` or `bytearray`) \[const char \*encoding, char \*\*buffer\]  
Same as `es` except that byte string objects are passed through without
recoding them. Instead, the implementation assumes that the byte string
object uses the encoding passed in as parameter.

`es#` (`str`) \[const char \*encoding, char \*\*buffer, :c`Py_ssize_t` \*buffer_length\]  
This variant on `s#` is used for encoding Unicode into a character
buffer. Unlike the `es` format, this variant allows input data which
contains NUL characters.

It requires three arguments. The first is only used as input, and must
be a :c`const char*` which points to the name of an encoding as a
NUL-terminated string, or `NULL`, in which case `'utf-8'` encoding is
used. An exception is raised if the named encoding is not known to
Python. The second argument must be a :c`char**`; the value of the
pointer it references will be set to a buffer with the contents of the
argument text. The text will be encoded in the encoding specified by the
first argument. The third argument must be a pointer to an integer; the
referenced integer will be set to the number of bytes in the output
buffer.

There are two modes of operation:

If *\*buffer* points a `NULL` pointer, the function will allocate a
buffer of the needed size, copy the encoded data into this buffer and
set *\*buffer* to reference the newly allocated storage. The caller is
responsible for calling :c`PyMem_Free` to free the allocated buffer
after usage.

If *\*buffer* points to a non-`NULL` pointer (an already allocated
buffer), :c`PyArg_ParseTuple` will use this location as the buffer and
interpret the initial value of *\*buffer_length* as the buffer size. It
will then copy the encoded data into the buffer and NUL-terminate it. If
the buffer is not large enough, a `ValueError` will be set.

In both cases, *\*buffer_length* is set to the length of the encoded
data without the trailing NUL byte.

`et#` (`str`, `bytes` or `bytearray`) \[const char \*encoding, char \*\*buffer, :c`Py_ssize_t` \*buffer_length\]  
Same as `es#` except that byte string objects are passed through without
recoding them. Instead, the implementation assumes that the byte string
object uses the encoding passed in as parameter.

3.12 `u`, `u#`, `Z`, and `Z#` are removed because they used a legacy
`Py_UNICODE*` representation.

### Numbers

`b` (`int`) \[unsigned char\]  
Convert a nonnegative Python integer to an unsigned tiny int, stored in
a C :c`unsigned char`.

`B` (`int`) \[unsigned char\]  
Convert a Python integer to a tiny int without overflow checking, stored
in a C :c`unsigned char`.

`h` (`int`) \[short int\]  
Convert a Python integer to a C :c`short int`.

`H` (`int`) \[unsigned short int\]  
Convert a Python integer to a C :c`unsigned short int`, without overflow
checking.

`i` (`int`) \[int\]  
Convert a Python integer to a plain C :c`int`.

`I` (`int`) \[unsigned int\]  
Convert a Python integer to a C :c`unsigned int`, without overflow
checking.

`l` (`int`) \[long int\]  
Convert a Python integer to a C :c`long int`.

`k` (`int`) \[unsigned long\]  
Convert a Python integer to a C :c`unsigned long` without overflow
checking.

`L` (`int`) \[long long\]  
Convert a Python integer to a C :c`long long`.

`K` (`int`) \[unsigned long long\]  
Convert a Python integer to a C :c`unsigned long long` without overflow
checking.

`n` (`int`) \[:c`Py_ssize_t`\]  
Convert a Python integer to a C :c`Py_ssize_t`.

`c` (`bytes` or `bytearray` of length 1) \[char\]  
Convert a Python byte, represented as a `bytes` or `bytearray` object of
length 1, to a C :c`char`.

3.3 Allow `bytearray` objects.

`C` (`str` of length 1) \[int\]  
Convert a Python character, represented as a `str` object of length 1,
to a C :c`int`.

`f` (`float`) \[float\]  
Convert a Python floating point number to a C :c`float`.

`d` (`float`) \[double\]  
Convert a Python floating point number to a C :c`double`.

`D` (`complex`) \[Py_complex\]  
Convert a Python complex number to a C :c`Py_complex` structure.

### Other objects

`O` (object) \[PyObject \*\]  
Store a Python object (without any conversion) in a C object pointer.
The C program thus receives the actual object that was passed. The
object's reference count is not increased. The pointer stored is not
`NULL`.

`O!` (object) \[*typeobject*, PyObject \*\]  
Store a Python object in a C object pointer. This is similar to `O`, but
takes two C arguments: the first is the address of a Python type object,
the second is the address of the C variable (of type :c`PyObject*`) into
which the object pointer is stored. If the Python object does not have
the required type, `TypeError` is raised.

`O&` (object) \[*converter*, *anything*\]  
Convert a Python object to a C variable through a *converter* function.
This takes two arguments: the first is a function, the second is the
address of a C variable (of arbitrary type), converted to :c`void *`.
The *converter* function in turn is called as follows:

``` c
status = converter(object, address);
```

where *object* is the Python object to be converted and *address* is the
:c`void*` argument that was passed to the :c`PyArg_Parse\*` function.
The returned *status* should be `1` for a successful conversion and `0`
if the conversion has failed. When the conversion fails, the *converter*
function should raise an exception and leave the content of *address*
unmodified.

If the *converter* returns `Py_CLEANUP_SUPPORTED`, it may get called a
second time if the argument parsing eventually fails, giving the
converter a chance to release any memory that it had already allocated.
In this second call, the *object* parameter will be `NULL`; *address*
will have the same value as in the original call.

3.1 `Py_CLEANUP_SUPPORTED` was added.

`p` (`bool`) \[int\]  
Tests the value passed in for truth (a boolean **p**redicate) and
converts the result to its equivalent C true/false integer value. Sets
the int to `1` if the expression was true and `0` if it was false. This
accepts any valid Python value. See `truth` for more information about
how Python tests values for truth.

3.3

`(items)` (`tuple`) \[*matching-items*\]  
The object must be a Python sequence whose length is the number of
format units in *items*. The C arguments must correspond to the
individual format units in *items*. Format units for sequences may be
nested.

It is possible to pass "long" integers (integers whose value exceeds the
platform's `LONG_MAX`) however no proper range checking is done --- the
most significant bits are silently truncated when the receiving field is
too small to receive the value (actually, the semantics are inherited
from downcasts in C --- your mileage may vary).

A few other characters have a meaning in a format string. These may not
occur inside nested parentheses. They are:

`|`  
Indicates that the remaining arguments in the Python argument list are
optional. The C variables corresponding to optional arguments should be
initialized to their default value --- when an optional argument is not
specified, :c`PyArg_ParseTuple` does not touch the contents of the
corresponding C variable(s).

`$`  
:c`PyArg_ParseTupleAndKeywords` only: Indicates that the remaining
arguments in the Python argument list are keyword-only. Currently, all
keyword-only arguments must also be optional arguments, so `|` must
always be specified before `$` in the format string.

3.3

`:`  
The list of format units ends here; the string after the colon is used
as the function name in error messages (the "associated value" of the
exception that :c`PyArg_ParseTuple` raises).

`;`  
The list of format units ends here; the string after the semicolon is
used as the error message *instead* of the default error message. `:`
and `;` mutually exclude each other.

Note that any Python object references which are provided to the caller
are *borrowed* references; do not decrement their reference count!

Additional arguments passed to these functions must be addresses of
variables whose type is determined by the format string; these are used
to store values from the input tuple. There are a few cases, as
described in the list of format units above, where these parameters are
used as input values; they should match what is specified for the
corresponding format unit in that case.

For the conversion to succeed, the *arg* object must match the format
and the format must be exhausted. On success, the :c`PyArg_Parse\*`
functions return true, otherwise they return false and raise an
appropriate exception. When the :c`PyArg_Parse\*` functions fail due to
conversion failure in one of the format units, the variables at the
addresses corresponding to that and the following format units are left
untouched.

### API Functions

> Parse the parameters of a function that takes only positional
> parameters into local variables. Returns true on success; on failure,
> it returns false and raises the appropriate exception.

> Identical to :c`PyArg_ParseTuple`, except that it accepts a va_list
> rather than a variable number of arguments.

> Parse the parameters of a function that takes both positional and
> keyword parameters into local variables. The *keywords* argument is a
> `NULL`-terminated array of keyword parameter names. Empty names denote
> `positional-only parameters <positional-only_parameter>`. Returns true
> on success; on failure, it returns false and raises the appropriate
> exception.
>
> 3.6 Added support for `positional-only parameters
> <positional-only_parameter>`.

> Identical to :c`PyArg_ParseTupleAndKeywords`, except that it accepts a
> va_list rather than a variable number of arguments.

> Ensure that the keys in the keywords argument dictionary are strings.
> This is only needed if :c`PyArg_ParseTupleAndKeywords` is not used,
> since the latter already does this check.
>
> 3.2

> Function used to deconstruct the argument lists of "old-style"
> functions ---these are functions which use the `METH_OLDARGS`
> parameter parsing method, which has been removed in Python 3. This is
> not recommended for use in parameter parsing in new code, and most
> code in the standard interpreter has been modified to no longer use
> this for that purpose. It does remain a convenient way to decompose
> other tuples, however, and may continue to be used for that purpose.

> A simpler form of parameter retrieval which does not use a format
> string to specify the types of the arguments. Functions which use this
> method to retrieve their parameters should be declared as
> `METH_VARARGS` in function or method tables. The tuple containing the
> actual parameters should be passed as *args*; it must actually be a
> tuple. The length of the tuple must be at least *min* and no more than
> *max*; *min* and *max* may be equal. Additional arguments must be
> passed to the function, each of which should be a pointer to a
> :c`PyObject*` variable; these will be filled in with the values from
> *args*; they will contain `borrowed references <borrowed reference>`.
> The variables which correspond to optional parameters not given by
> *args* will not be filled in; these should be initialized by the
> caller. This function returns true on success and false if *args* is
> not a tuple or contains the wrong number of elements; an exception
> will be set if there was a failure.
>
> This is an example of the use of this function, taken from the sources
> for the `_weakref` helper module for weak references:
>
> ``` c
> static PyObject *
> weakref_ref(PyObject *self, PyObject *args)
> {
>     PyObject *object;
>     PyObject *callback = NULL;
>     PyObject *result = NULL;
>
>     if (PyArg_UnpackTuple(args, "ref", 1, 2, &object, &callback)) {
>         result = PyWeakref_NewRef(object, callback);
>     }
>     return result;
> }
> ```
>
> The call to :c`PyArg_UnpackTuple` in this example is entirely
> equivalent to this call to :c`PyArg_ParseTuple`:
>
> ``` c
> PyArg_ParseTuple(args, "O|O:ref", &object, &callback)
> ```

## Building values

> Create a new value based on a format string similar to those accepted
> by the :c`PyArg_Parse\*` family of functions and a sequence of values.
> Returns the value or `NULL` in the case of an error; an exception will
> be raised if `NULL` is returned.
>
> :c`Py_BuildValue` does not always build a tuple. It builds a tuple
> only if its format string contains two or more format units. If the
> format string is empty, it returns `None`; if it contains exactly one
> format unit, it returns whatever object is described by that format
> unit. To force it to return a tuple of size 0 or one, parenthesize the
> format string.
>
> When memory buffers are passed as parameters to supply data to build
> objects, as for the `s` and `s#` formats, the required data is copied.
> Buffers provided by the caller are never referenced by the objects
> created by :c`Py_BuildValue`. In other words, if your code invokes
> :c`malloc` and passes the allocated memory to :c`Py_BuildValue`, your
> code is responsible for calling :c`free` for that memory once
> :c`Py_BuildValue` returns.
>
> In the following description, the quoted form is the format unit; the
> entry in (round) parentheses is the Python object type that the format
> unit will return; and the entry in \[square\] brackets is the type of
> the C value(s) to be passed.
>
> The characters space, tab, colon and comma are ignored in format
> strings (but not within format units such as `s#`). This can be used
> to make long format strings a tad more readable.
>
> `s` (`str` or `None`) \[const char \*\]  
> Convert a null-terminated C string to a Python `str` object using
> `'utf-8'` encoding. If the C string pointer is `NULL`, `None` is used.
>
> `s#` (`str` or `None`) \[const char \*, :c`Py_ssize_t`\]  
> Convert a C string and its length to a Python `str` object using
> `'utf-8'` encoding. If the C string pointer is `NULL`, the length is
> ignored and `None` is returned.
>
> `y` (`bytes`) \[const char \*\]  
> This converts a C string to a Python `bytes` object. If the C string
> pointer is `NULL`, `None` is returned.
>
> `y#` (`bytes`) \[const char \*, :c`Py_ssize_t`\]  
> This converts a C string and its lengths to a Python object. If the C
> string pointer is `NULL`, `None` is returned.
>
> `z` (`str` or `None`) \[const char \*\]  
> Same as `s`.
>
> `z#` (`str` or `None`) \[const char \*, :c`Py_ssize_t`\]  
> Same as `s#`.
>
> `u` (`str`) \[const wchar_t \*\]  
> Convert a null-terminated :c`wchar_t` buffer of Unicode (UTF-16 or
> UCS-4) data to a Python Unicode object. If the Unicode buffer pointer
> is `NULL`, `None` is returned.
>
> `u#` (`str`) \[const wchar_t \*, :c`Py_ssize_t`\]  
> Convert a Unicode (UTF-16 or UCS-4) data buffer and its length to a
> Python Unicode object. If the Unicode buffer pointer is `NULL`, the
> length is ignored and `None` is returned.
>
> `U` (`str` or `None`) \[const char \*\]  
> Same as `s`.
>
> `U#` (`str` or `None`) \[const char \*, :c`Py_ssize_t`\]  
> Same as `s#`.
>
> `i` (`int`) \[int\]  
> Convert a plain C :c`int` to a Python integer object.
>
> `b` (`int`) \[char\]  
> Convert a plain C :c`char` to a Python integer object.
>
> `h` (`int`) \[short int\]  
> Convert a plain C :c`short int` to a Python integer object.
>
> `l` (`int`) \[long int\]  
> Convert a C :c`long int` to a Python integer object.
>
> `B` (`int`) \[unsigned char\]  
> Convert a C :c`unsigned char` to a Python integer object.
>
> `H` (`int`) \[unsigned short int\]  
> Convert a C :c`unsigned short int` to a Python integer object.
>
> `I` (`int`) \[unsigned int\]  
> Convert a C :c`unsigned int` to a Python integer object.
>
> `k` (`int`) \[unsigned long\]  
> Convert a C :c`unsigned long` to a Python integer object.
>
> `L` (`int`) \[long long\]  
> Convert a C :c`long long` to a Python integer object.
>
> `K` (`int`) \[unsigned long long\]  
> Convert a C :c`unsigned long long` to a Python integer object.
>
> `n` (`int`) \[:c`Py_ssize_t`\]  
> Convert a C :c`Py_ssize_t` to a Python integer.
>
> `c` (`bytes` of length 1) \[char\]  
> Convert a C :c`int` representing a byte to a Python `bytes` object of
> length 1.
>
> `C` (`str` of length 1) \[int\]  
> Convert a C :c`int` representing a character to Python `str` object of
> length 1.
>
> `d` (`float`) \[double\]  
> Convert a C :c`double` to a Python floating point number.
>
> `f` (`float`) \[float\]  
> Convert a C :c`float` to a Python floating point number.
>
> `D` (`complex`) \[Py_complex \*\]  
> Convert a C :c`Py_complex` structure to a Python complex number.
>
> `O` (object) \[PyObject \*\]  
> Pass a Python object untouched (except for its reference count, which
> is incremented by one). If the object passed in is a `NULL` pointer,
> it is assumed that this was caused because the call producing the
> argument found an error and set an exception. Therefore,
> :c`Py_BuildValue` will return `NULL` but won't raise an exception. If
> no exception has been raised yet, `SystemError` is set.
>
> `S` (object) \[PyObject \*\]  
> Same as `O`.
>
> `N` (object) \[PyObject \*\]  
> Same as `O`, except it doesn't increment the reference count on the
> object. Useful when the object is created by a call to an object
> constructor in the argument list.
>
> `O&` (object) \[*converter*, *anything*\]  
> Convert *anything* to a Python object through a *converter* function.
> The function is called with *anything* (which should be compatible
> with :c`void*`) as its argument and should return a "new" Python
> object, or `NULL` if an error occurred.
>
> `(items)` (`tuple`) \[*matching-items*\]  
> Convert a sequence of C values to a Python tuple with the same number
> of items.
>
> `[items]` (`list`) \[*matching-items*\]  
> Convert a sequence of C values to a Python list with the same number
> of items.
>
> `{items}` (`dict`) \[*matching-items*\]  
> Convert a sequence of C values to a Python dictionary. Each pair of
> consecutive C values adds one item to the dictionary, serving as key
> and value, respectively.
>
> If there is an error in the format string, the `SystemError` exception
> is set and `NULL` returned.

> Identical to :c`Py_BuildValue`, except that it accepts a va_list
> rather than a variable number of arguments.